# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [ ]:
%load_ext dotenv
%dotenv

In [2]:
import dask.dataframe as dd

c:\Users\Caro Crespo\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:15: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [8]:
import os
from glob import glob

PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files_path = glob(os.path.join(os.getenv('PRICE_DATA'), "**/*.parquet"), recursive = True)

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [22]:
dd_px = dd.read_parquet(parquet_files_path).set_index("ticker")
dd_shift = dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1), 
                       Adj_Close_lag_1 = x['Adj Close'].shift(1))
)
dd_rets = dd_shift.assign(
    Returns = lambda x: x['Close']/x['Close_lag_1'] - 1
)
dd_feat = dd_rets.assign(
    hi_lo_range = lambda x: x['High']-x['Low']
)

C:\Users\Caro Crespo\AppData\Local\Temp\ipykernel_764\2354929237.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = dd_px.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [61]:
pd_feat = dd_feat.compute()

In [62]:
pd_feat.groupby('ticker').apply(
    lambda x : x.assign(moving_average=x['Returns'].rolling(10).mean())
)

Date   Open   High    Low  Close  Adj Close      Volume  \
ticker ticker                                                                 
ACN    ACN    2001-07-19  15.10  15.29  15.00  15.17  11.404394  34994300.0   
       ACN    2001-07-20  15.05  15.05  14.80  15.01  11.284108   9238500.0   
       ACN    2001-07-23  15.00  15.01  14.55  15.00  11.276587   7501000.0   
       ACN    2001-07-24  14.95  14.97  14.70  14.86  11.171341   3537300.0   
       ACN    2001-07-25  14.70  14.95  14.65  14.95  11.238999   4208100.0   
...                  ...    ...    ...    ...    ...        ...         ...   
ZIXI   ZIXI   2003-06-26   4.04   4.19   3.86   4.00   4.000000    515300.0   
       ZIXI   2003-06-27   4.00   4.05   3.79   3.85   3.850000    162400.0   
       ZIXI   2003-06-30   3.84   4.00   3.72   3.77   3.770000    119900.0   
       ZIXI   2003-07-01   3.72   3.85   3.65   3.78   3.780000    202100.0   
       ZIXI   2003-07-02   3.81   3.99   3.73   3.73   3.730000     81100.0   

                 source  Year  Close_lag_1  Adj_Close_lag_1   Returns  \
ticker ticker                                                           
ACN    ACN      ACN.csv  2001          NaN              NaN       NaN   
       ACN      ACN.csv  2001        15.17        11.404394 -0.010547   
       ACN      ACN.csv  2001        15.01        11.284108 -0.000666   
       ACN      ACN.csv  2001        15.00        11.276587 -0.009333   
       ACN      ACN.csv  2001        14.86        11.171341  0.006057   
...                 ...   ...          ...              ...       ...   
ZIXI   ZIXI    ZIXI.csv  2003         4.04         4.040000 -0.009901   
       ZIXI    ZIXI.csv  2003         4.00         4.000000 -0.037500   
       ZIXI    ZIXI.csv  2003         3.85         3.850000 -0.020779   
       ZIXI    ZIXI.csv  2003         3.77         3.770000  0.002653   
       ZIXI    ZIXI.csv  2003         3.78         3.780000 -0.013228   

               hi_lo_range  moving_average  
ticker ticker                               
ACN    ACN            0.29             NaN  
       ACN            0.25             NaN  
       ACN            0.46             NaN  
       ACN            0.27             NaN  
       ACN            0.30             NaN  
...                    ...             ...  
ZIXI   ZIXI           0.33       -0.006616  
       ZIXI           0.26       -0.011064  
       ZIXI           0.28       -0.014528  
       ZIXI           0.20       -0.016768  
       ZIXI           0.26       -0.015646  

[239659 rows x 14 columns]

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

In [ ]:
# It wasn't necessary, it could be done with the Dask dataframe
# In this case, I don't think so, because the panda dataframe didn't have so many raws.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.